In [1]:
%matplotlib inline

Now I need to do my imports

In [2]:
from utils import *
from vgg16 import Vgg16

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Quadro M4000 (CNMeM is disabled, cuDNN 5110)
Using Theano backend.


In [3]:
batch_size = 64

Set up paths

In [18]:
sample_path = "data/statefarm/sample/"

#path = sample_path
path = "data/statefarm/"

training_path = path + 'train'
validation_path = path + 'valid'
test_path = path + 'test'
results_path = path + 'results'
submission_path = path + 'submission'  #For kaggle comps

Set up working paths

In [6]:
# commands go here
# Can I turn this into a util function

print('Current working dir: %s') %os.getcwd()

current_dir = os.getcwd()
LESSON_HOME_DIR = current_dir
DATA_HOME_DIR = current_dir+'/data/statefarm'


Current working dir: /home/paperspace/Projects/courses/deeplearning1/nbs


Create directories

In [8]:
%cd $DATA_HOME_DIR
%mkdir valid
%mkdir results
%mkdir submission
%mkdir -p sample/train
%mkdir -p sample/test
%mkdir -p sample/valid
%mkdir -p sample/results
%mkdir -p sample/submission
%mkdir -p test/unknown

/home/paperspace/Projects/courses/deeplearning1/nbs/data/statefarm
mkdir: cannot create directory ‘valid’: File exists
mkdir: cannot create directory ‘results’: File exists
mkdir: cannot create directory ‘submission’: File exists


Build validation set

Copy files from train to valid

In [9]:
%cd $DATA_HOME_DIR/train

/home/paperspace/Projects/courses/deeplearning1/nbs/data/statefarm/train


In [10]:
for d in glob('c?'):
    os.mkdir('../valid/'+d)

g = glob('c?/*.jpg')
shuf = np.random.permutation(g)
for i in range(2000): os.rename(shuf[i], DATA_HOME_DIR+'/valid/' + shuf[i])

In [11]:
from shutil import copyfile

Create directories in sample for each of the categories

In [13]:
for d in glob('c?'):
    os.mkdir('../sample/train/'+d)
    os.mkdir('../sample/valid/'+d)

In [14]:
g = glob('c?/*.jpg')
shuf = np.random.permutation(g)
for i in range(1500): copyfile(shuf[i], '../sample/train/' + shuf[i])

In [15]:
%cd ../valid

/home/paperspace/Projects/courses/deeplearning1/nbs/data/statefarm/valid


In [16]:
g = glob('c?/*.jpg')
shuf = np.random.permutation(g)
for i in range(1000): copyfile(shuf[i], '../sample/valid/' + shuf[i])

In [17]:
%cd ../../..

/home/paperspace/Projects/courses/deeplearning1/nbs


Now set up the model

In [19]:
vgg = Vgg16()
batches = vgg.get_batches(training_path, batch_size=batch_size)
val_batches = vgg.get_batches(validation_path, batch_size=batch_size*2)
vgg.finetune(batches)

Found 20424 images belonging to 10 classes.
Found 2000 images belonging to 10 classes.


In [20]:
num_epochs = 5

In [22]:
for epoch_num in range(num_epochs):
    print('Running epoch: %i') % epoch_num
    vgg.fit(batches, val_batches, nb_epoch=1)
    latest_weights_filename = 'ft%d.h5' % epoch_num
    vgg.model.save_weights(results_path+'/'+latest_weights_filename)
print("Completed %d epochs!") %num_epochs

Running epoch: 0
Epoch 1/1
20424/20424 [==============================] - 513s - loss: 1.0694 - acc: 0.6627 - val_loss: 0.3702 - val_acc: 0.8965
Running epoch: 1
Epoch 1/1
20424/20424 [==============================] - 518s - loss: 0.9351 - acc: 0.7018 - val_loss: 0.2892 - val_acc: 0.9275
Running epoch: 2
Epoch 1/1
20424/20424 [==============================] - 514s - loss: 0.8820 - acc: 0.7175 - val_loss: 0.2880 - val_acc: 0.9200
Running epoch: 3
Epoch 1/1
20424/20424 [==============================] - 514s - loss: 0.8690 - acc: 0.7260 - val_loss: 0.2685 - val_acc: 0.9235
Running epoch: 4
Epoch 1/1
20424/20424 [==============================] - 513s - loss: 0.8773 - acc: 0.7283 - val_loss: 0.2216 - val_acc: 0.9380
Completed 5 epochs!


In [25]:
batches, predictions = vgg.test(test_path+'/', batch_size=batch_size*2)

Found 79726 images belonging to 1 classes.


In [27]:
print(predictions[:10])

[[  1.7160e-02   1.6687e-04   4.9131e-01   3.1354e-04   5.9125e-03   5.4914e-03   4.2073e-01
    6.7505e-04   5.6945e-02   1.2886e-03]
 [  2.0337e-04   9.6896e-02   2.0614e-03   7.6486e-03   5.4816e-02   1.5609e-02   8.0979e-02
    3.7838e-01   3.2753e-01   3.5884e-02]
 [  6.8442e-03   2.6620e-03   5.9042e-02   5.8311e-03   2.0003e-01   4.9203e-02   5.3324e-01
    1.9789e-05   7.1470e-02   7.1658e-02]
 [  3.9611e-01   1.1534e-01   3.7099e-04   1.3818e-01   8.9354e-02   1.7916e-01   3.4354e-03
    1.2404e-04   2.4930e-02   5.2995e-02]
 [  1.3958e-02   2.2939e-01   7.7875e-05   1.5845e-02   2.8932e-04   6.9197e-01   3.3709e-02
    4.7598e-04   2.5911e-03   1.1695e-02]
 [  2.2241e-03   3.1662e-02   4.4102e-03   5.5165e-01   1.2102e-01   2.7062e-02   2.2046e-01
    1.2675e-03   3.9964e-02   2.8597e-04]
 [  1.2850e-04   2.2435e-05   6.5521e-06   7.1047e-06   3.6648e-04   9.7228e-02   1.5854e-05
    8.9517e-06   1.3923e-02   8.8829e-01]
 [  5.3737e-03   2.6260e-03   3.6020e-06   9.7257e-01  

Hold off on this part until I understand the submission format

img, c0, c1, ... c9

In [28]:
filenames = batches.filenames
print(filenames[:10])

['unknown/img_64880.jpg', 'unknown/img_14880.jpg', 'unknown/img_51085.jpg', 'unknown/img_4678.jpg', 'unknown/img_65918.jpg', 'unknown/img_41634.jpg', 'unknown/img_46989.jpg', 'unknown/img_45134.jpg', 'unknown/img_64717.jpg', 'unknown/img_24311.jpg']


In [30]:
submission_file_name = 'submission1.csv'

#pred_sub = predictions.clip(min=0.01, max=0.95)

filenames = batches.filenames
ids = np.array([f[8:] for f in filenames])

subm = np.stack([ids,predictions], axis=1)

np.savetxt(results_path + '/' + submission_file_name, subm, 
           fmt='%d,%.5f,%.5f,%.5f,%.5f,%.5f,%.5f,%.5f,%.5f,%.5f,%.5f', 
           header='img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9', comments='')

ValueError: all input arrays must have the same shape

In [25]:
%pwd

u'/home/paperspace/Projects/courses/deeplearning1/nbs'

Five epoch training got me to Rank 340 / 1314